In [1]:
from bs4 import BeautifulSoup
import asyncio
from aiohttp import ClientSession
import numpy as np
import pandas as pd
from functools import partial
from aiohttp import ClientConnectorError
import json
import re
from bs4 import NavigableString
from pprint import pprint
from datetime import datetime, timedelta
import os
from itertools import chain
import pickle

In [2]:
trails = pickle.load(open("./data/mondo_scraped/cleaned_int_sp.pkl", "rb"))

In [4]:
trails[:10]

['http://www.elmundo.es/deportes/2016/02/12/56bd9f7222601dec3d8b4677.html',
 'http://www.elmundo.es/deportes/mas-deporte/2017/08/04/5983777ce5fdea972d8b45d0.html',
 'http://www.elmundo.es/deportes/2015/08/20/55d5ea94ca4741023b8b4594.html',
 'http://www.elmundo.es/deportes/2016/08/21/57b8d2b3e5fdea111c8b458f.html',
 'http://www.elmundo.es/comunidad-valenciana/2014/04/12/53497f89268e3e0c6a8b4575.html',
 'http://www.elmundo.es/deportes/2014/01/26/52e5540be2704ede598b4576.html',
 'http://www.elmundo.es/deportes/2017/02/22/58adead8e5fdea446a8b464a.html',
 'http://www.elmundo.es/deportes/2016/04/27/5720a3eee2704e8e068b463f.html',
 'http://www.elmundo.es/deportes/2015/12/17/5672cbc846163f76248b459f.html',
 'http://www.elmundo.es/deportes/2016/12/17/58555a5be2704ec66f8b460d.html']

In [5]:
len(trails)

15290

In [6]:
async def get_html_data(trail, session):
    """Access El Mondo webpage"""
    url = trail
    attempts = 0
    while attempts < 3:
        try:
            res = await session.get(url)
            try:
                html = await res.text()
            except UnicodeDecodeError:
                return trail, None
            try:
                soup = BeautifulSoup(html, 'html.parser')
                headline = soup.select("h1[itemprop='headline']")[0].text
                facts = [fact.text for fact in soup.select("p.summary-lead, h2[itemprop='alternativeHeadline']")]
                body = "\n".join([sent.text for sent in soup.select("div[itemprop='articleBody']")[0]
                                 .find_all('p', class_=lambda x: x != 'summary-lead')])
                tag = [tag.text for tag in soup.select("ul.item-tags a")]
            except (IndexError, TypeError) as e:
                global DEBUG
                DEBUG.append((trail, e, soup))
                print("content error!")
                return trail, None
            try:
                time = soup.select("time")[0]['datetime']
            except:
                time = None
            attrs = {
                "headline": headline,
                "keyfacts": facts, 
                "content": body,
                "tags": tag,
                "time": time
                    }
            return trail, attrs
        except ClientConnectorError:
            attempts += 1
            print("Connector error occurred!")
    if attempts == 3:
        return trail, None
    
async def gather_results(curr, step, trails):
    """Launch scrape tasks and collect results"""
    tasks = []
    async with ClientSession() as session:
        for trail in trails[curr: curr + step]:
            task = asyncio.ensure_future(get_html_data(trail, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses


def process_df(future, curr, step):
    """Save scrape results in json files"""
    cache = {k: v for k, v in future.result()}
#     cache = future.result()
    if len(cache) == 0:
        raise RuntimeError("Empty response!")
    else:
#         json.dump(cache, open("./data/scraped/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        json.dump(cache, open("./data/mondo_scraped/contents_sp/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        print("got it! ({0}, {1})".format(curr, curr + step))

In [17]:
# df = pd.read_csv("./data/ratings.csv")
# app_ids = df.loc[:, "app_id"].astype(str)

start = 14900
# end = len(trails)
step = 100
end = start + step
DEBUG = []

for curr in range(start, end, step):
    print("loading data from {0} to {1}".format(curr, curr + step))
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(gather_results(curr, step, trails))
    future.add_done_callback(partial(process_df, curr=curr, step=step))
    loop.run_until_complete(future)

loading data from 14900 to 15000
got it! (14900, 15000)


200, 1100, 2300, 4100, 6800, 8000, 8700, 10700, 14900

## convert

In [18]:
doc_list = os.listdir("./data/mondo_scraped/contents_sp/")
articles = dict()
for file in doc_list:
    if file[-4:] == 'json':
        with open(os.path.join("./data/mondo_scraped/contents_sp/", file), "r") as f:
            articles.update(json.load(f))

len(articles)

15290

In [19]:
clean = {k: v for k, v in articles.items() if v is not None}

In [20]:
len(clean)

15200

In [21]:
docs = pd.DataFrame.from_dict(clean, orient='index')

In [22]:
valids = docs.loc[docs['keyfacts'].astype(np.bool), :]
valids.shape

(11580, 5)

In [23]:
valids.head()

,headline,keyfacts,content,tags,time
http://www.elmundo.es//2016/08/31/57c6baad46163f610d8b45b3.html,"Rosberg: ""Estoy disfrutando mucho de la lucha ...",[El piloto germano alaba el gran ambiente de l...,El piloto alemán Nico Rosberg (Mercedes) esper...,"[fórmula 1, Nico Rosberg, Lewis Hamilton, depo...",2016-08-31 13:12:13
http://www.elmundo.es/america/2014/12/29/54a19293e2704e19708b458f.html,El estadio de David Beckham en Miami se esfuma,[\nEl futbolista quiere construirlo en una zon...,El sueño del astro David Beckham se ha aplazad...,[],2014-12-29
http://www.elmundo.es/andalucia/2014/01/03/52c69fe322601dd86c8b4576.html,Del Nido recogerá firmas en apoyo de su solici...,[ Voluntarios recorrerán los aledaños del Sánc...,"La mañana del domingo 5 de enero, unos 40 volu...",[],2014-01-03
http://www.elmundo.es/andalucia/2014/01/14/52d54c02268e3eb1318b4579.html,Garrido va pidiendo la cuenta,[ El entrenador del Betis critica a la directi...,"Nada más aterrizar Juan Carlos Garrido, en el ...",[],2014-01-14
http://www.elmundo.es/andalucia/2014/01/19/52dc119eca4741a5458b4570.html,El keniano Paul Tanui inscribe su nombre en el...,[ Triunfo para la etíope Hiwot Ayalew en la ca...,El keniano Paul Tanui ha inscrito este domingo...,[],2014-01-19


In [24]:
valids.to_pickle('./data/elmondo_es_sp.pkl')

In [25]:
valids.iloc[7, :].apply(print);

'Trabajé en el patio de la prisión de Lleida con el Vaquilla, tras 14 años en una cárcel lo relativizas todo'
[' EL MUNDO prepara la Copa del Rey con el entrenador anfitrión en el mítico Pimpi Florida ', ' "Si la Liga no es más seria se nos irá de las manos" ', ' "El Real Madrid tiene la confianza para llegar a cotas que ni ellos mismo esperan" ', ' "Barcelona, como otras veces Madrid, Unicaja o Laboral Kutxa, está rehaciendo lo andado" ', ' "O mejoramos todos la calidad del producto o será difícil que esto no se convierta en una Liga menor" ', " Un 'beatle' en el templo de la Copa "]
En las paredes, plantillas añejas del Unicaja, entre fotos de folclóricas, Garbajosa con el fisio de los Toronto Raptors o guiños de la selección a un taberna que huele a mar, vino, copla, Beatles y baloncesto. Joan Plaza se encontró con un santuario: El Pimpi Florida.
Pregunta.-Le hemos traído aquí y confiesa que no bebe...
Respuesta.-Nunca había bebido whisky hasta que me nombran entrenador del Madrid, 